# Implementing logistic regression from scratch

The goal of this notebook is to implement your own logistic regression classifier. You will:

 * Extract features from Amazon product reviews.
 * Convert an SFrame into a NumPy array.
 * Implement the link function for logistic regression.
 * Write a function to compute the derivative of the log likelihood function with respect to a single coefficient.
 * Implement gradient ascent.
 * Given a set of coefficients, predict sentiments.
 * Compute classification accuracy for the logistic regression model.
 
Let's get started!
    
## Fire up GraphLab Create

Make sure you have the latest version of GraphLab Create. Upgrade by
```
   pip install graphlab-create --upgrade
```
See [this page](https://dato.com/download/) for detailed instructions on upgrading.

In [1]:
import graphlab

## Load review dataset

For this assignment, we will use a subset of the Amazon product review dataset. The subset was chosen to contain similar numbers of positive and negative reviews, as the original dataset consisted primarily of positive reviews.

In [2]:
products = graphlab.SFrame('amazon_baby_subset.gl/')

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\qianc\AppData\Local\Temp\graphlab_server_1487250346.log.0


This non-commercial license of GraphLab Create for academic use is assigned to qianchao0928@foxmail.com and will expire on August 18, 2017.


In [5]:
products.head(3)

name,review,rating,sentiment
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,1
Nature's Lullabies SecondYear Sticker Calendar ...,We wanted to getsomething to keep track ...,5.0,1
Nature's Lullabies SecondYear Sticker Calendar ...,My daughter had her 1stbaby over a year ago. ...,5.0,1


One column of this dataset is 'sentiment', corresponding to the class label with +1 indicating a review with positive sentiment and -1 indicating one with negative sentiment.

In [6]:
products['sentiment']

dtype: int
Rows: 53072
[1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, ... ]

Let us quickly explore more of this dataset.  The 'name' column indicates the name of the product.  Here we list the first 10 products in the dataset.  We then count the number of positive and negative reviews.

In [7]:
products.head(10)['name']

dtype: str
Rows: 10
["Stop Pacifier Sucking without tears with Thumbuddy To Love's Binky Fairy Puppet and Adorable Book", "Nature's Lullabies Second Year Sticker Calendar", "Nature's Lullabies Second Year Sticker Calendar", 'Lamaze Peekaboo, I Love You', "SoftPlay Peek-A-Boo Where's Elmo A Children's Book", 'Our Baby Girl Memory Book', 'Hunnt&reg; Falling Flowers and Birds Kids Nursery Home Decor Vinyl Mural Art Wall Paper Stickers', 'Blessed By Pope Benedict XVI Divine Mercy Full Color Medal', 'Cloth Diaper Pins Stainless Steel Traditional Safety Pin (Black)', 'Cloth Diaper Pins Stainless Steel Traditional Safety Pin (Black)']

In [8]:
print '# of positive reviews =', len(products[products['sentiment']==1])
print '# of negative reviews =', len(products[products['sentiment']==-1])

# of positive reviews = 26579
# of negative reviews = 26493


**Note:** For this assignment, we eliminated class imbalance by choosing 
a subset of the data with a similar number of positive and negative reviews. 

## Apply text cleaning on the review data

In this section, we will perform some simple feature cleaning using **SFrames**. The last assignment used all words in building bag-of-words features, but here we limit ourselves to 193 words (for simplicity). We compiled a list of 193 most frequent words into a JSON file. 

Now, we will load these words from this JSON file:

In [9]:
import json
with open('important_words.json', 'r') as f: # Reads the list of most frequent words
    important_words = json.load(f)
important_words = [str(s) for s in important_words]

In [10]:
print important_words

['baby', 'one', 'great', 'love', 'use', 'would', 'like', 'easy', 'little', 'seat', 'old', 'well', 'get', 'also', 'really', 'son', 'time', 'bought', 'product', 'good', 'daughter', 'much', 'loves', 'stroller', 'put', 'months', 'car', 'still', 'back', 'used', 'recommend', 'first', 'even', 'perfect', 'nice', 'bag', 'two', 'using', 'got', 'fit', 'around', 'diaper', 'enough', 'month', 'price', 'go', 'could', 'soft', 'since', 'buy', 'room', 'works', 'made', 'child', 'keep', 'size', 'small', 'need', 'year', 'big', 'make', 'take', 'easily', 'think', 'crib', 'clean', 'way', 'quality', 'thing', 'better', 'without', 'set', 'new', 'every', 'cute', 'best', 'bottles', 'work', 'purchased', 'right', 'lot', 'side', 'happy', 'comfortable', 'toy', 'able', 'kids', 'bit', 'night', 'long', 'fits', 'see', 'us', 'another', 'play', 'day', 'money', 'monitor', 'tried', 'thought', 'never', 'item', 'hard', 'plastic', 'however', 'disappointed', 'reviews', 'something', 'going', 'pump', 'bottle', 'cup', 'waste', 'retu

Now, we will perform 2 simple data transformations:

1. Remove punctuation using [Python's built-in](https://docs.python.org/2/library/string.html) string functionality.
2. Compute word counts (only for **important_words**)

We start with *Step 1* which can be done as follows:

In [14]:
def remove_punctuation(text):
    import string
    return text.translate(None, string.punctuation)  
    # string.punctuation
    # '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

products['review_clean'] = products['review'].apply(remove_punctuation) # 删除标点符号

Now we proceed with *Step 2*. For each word in **important_words**, we compute a count for the number of times the word occurs in the review. We will store this count in a separate column (one for each word). The result of this feature processing is a single column for each word in **important_words** which keeps a count of the number of times the respective word occurs in the review text.


**Note:** There are several ways of doing this. In this assignment, we use the built-in *count* function for Python lists. Each review string is first split into individual words and the number of occurances of a given word is counted.

In [15]:
for word in important_words:
    products[word] = products['review_clean'].apply(lambda s : s.split().count(word))

In [17]:
products.head(2)

Columns:
	name	str
	review	str
	rating	float
	sentiment	int
	review_clean	str
	baby	int
	one	int
	great	int
	love	int
	use	int
	would	int
	like	int
	easy	int
	little	int
	seat	int
	old	int
	well	int
	get	int
	also	int
	really	int
	son	int
	time	int
	bought	int
	product	int
	good	int
	daughter	int
	much	int
	loves	int
	stroller	int
	put	int
	months	int
	car	int
	still	int
	back	int
	used	int
	recommend	int
	first	int
	even	int
	perfect	int
	nice	int
	bag	int
	two	int
	using	int
	got	int
	fit	int
	around	int
	diaper	int
	enough	int
	month	int
	price	int
	go	int
	could	int
	soft	int
	since	int
	buy	int
	room	int
	works	int
	made	int
	child	int
	keep	int
	size	int
	small	int
	need	int
	year	int
	big	int
	make	int
	take	int
	easily	int
	think	int
	crib	int
	clean	int
	way	int
	quality	int
	thing	int
	better	int
	without	int
	set	int
	new	int
	every	int
	cute	int
	best	int
	bottles	int
	work	int
	purchased	int
	right	int
	lot	int
	side	int
	happy	int
	comfortable	int
	toy	int
	able	int
	kids	int
	bit	int
	night	int
	long	int
	fits	int
	see	int
	us	int
	another	int
	play	int
	day	int
	money	int
	monitor	int
	tried	int
	thought	int
	never	int
	item	int
	hard	int
	plastic	int
	however	int
	disappointed	int
	reviews	int
	something	int
	going	int
	pump	int
	bottle	int
	cup	int
	waste	int
	return	int
	amazon	int
	different	int
	top	int
	want	int
	problem	int
	know	int
	water	int
	try	int
	received	int
	sure	int
	times	int
	chair	int
	find	int
	hold	int
	gate	int
	open	int
	bottom	int
	away	int
	actually	int
	cheap	int
	worked	int
	getting	int
	ordered	int
	came	int
	milk	int
	bad	int
	part	int
	worth	int
	found	int
	cover	int
	many	int
	design	int
	looking	int
	weeks	int
	say	int
	wanted	int
	look	int
	place	int
	purchase	int
	looks	int
	second	int
	piece	int
	box	int
	pretty	int
	trying	int
	difficult	int
	together	int
	though	int
	give	int
	started	int
	anything	int
	last	int
	company	int
	come	int
	returned	int
	maybe	int
	took	int
	broke	int
	makes	int
	stay	int
	instead	int
	idea	int
	head	int
	said	int
	less	int
	went	int
	working	int
	high	int
	unit	int
	seems	int
	picture	int
	completely	int
	wish	int
	buying	int
	babies	int
	won	int
	tub	int
	almost	int
	either	int

Rows: 2

Data:
+-------------------------------+-------------------------------+--------+-----------+
|              name             |             review            | rating | sentiment |
+-------------------------------+-------------------------------+--------+-----------+
| Stop Pacifier Sucking with... | All of my kids have cried ... |  5.0   |     1     |
| Nature's Lullabies Second ... | We wanted to get something... |  5.0   |     1     |
+-------------------------------+-------------------------------+--------+-----------+
+-------------------------------+------+-----+-------+------+-----+-------+------+
|          review_clean         | baby | one | great | love | use | would | like |
+-------------------------------+------+-----+-------+------+-----+-------+------+
| All of my kids have cried ... |  0   |  0  |   1   |  0   |  0  |   0   |  0   |
| We wanted to get something... |  0   |  0  |   0   |  0   |  0  |   0   |  0   |
+-------------------------------+------+-----+-------+------+-----+-------+------+
+------+--------+------+-----+------+-----+------+--------+-----+
| easy | little | seat | old | well | get | also | really | ... |
+------+--------+------+-----+------+-----+------+--------+-----+
|  1   |   0    |  0   |  0  |  0   |  0  |  0   |   0    | ... |
|  0   |   0    |  0   |  0  |  0   |  1  |  0   |   0    | ... |
+------+--------+------+-----+------+-----+------+--------+-----+
[2 rows x 198 columns]

The SFrame **products** now contains one column for each of the 193 **important_words**. As an example, the column **perfect** contains a count of the number of times the word **perfect** occurs in each of the reviews.

In [18]:
products['perfect']

dtype: int
Rows: 53072
[0L, 0L, 0L, 1L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 1L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 1L, 0L, 1L, 0L, 0L, 1L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, ... ]

In [26]:
products['perfect'][products['perfect']>=1].shape

(2955,)

Now, write some code to compute the number of product reviews that contain the word **perfect**.

**Hint**: 
* First create a column called `contains_perfect` which is set to 1 if the count of the word **perfect** (stored in column **perfect**) is >= 1.
* Sum the number of 1s in the column `contains_perfect`.

In [20]:
products['contains_perfect'] = products['perfect'].apply(lambda x : 1 if x >=1 else 0)

**Quiz Question**. How many reviews contain the word **perfect**?

In [22]:
products['contains_perfect'].sum()

2955L

## Convert SFrame to NumPy array

As you have seen previously, NumPy is a powerful library for doing matrix manipulation. Let us convert our data to matrices and then implement our algorithms with matrices.

First, make sure you can perform the following import.

In [33]:
import numpy as np
import math

We now provide you with a function that extracts columns from an SFrame and converts them into a NumPy array. Two arrays are returned: one representing features and another representing class labels. Note that the feature matrix includes an additional column 'intercept' to take account of the intercept term.

In [28]:
def get_numpy_data(data_sframe, features, label):
    data_sframe['intercept'] = 1
    features = ['intercept'] + features
    features_sframe = data_sframe[features]
    feature_matrix = features_sframe.to_numpy()
    label_sarray = data_sframe[label]
    label_array = label_sarray.to_numpy()
    return(feature_matrix, label_array)

Let us convert the data into NumPy arrays.

In [29]:
# Warning: This may take a few minutes...
feature_matrix, sentiment = get_numpy_data(products, important_words, 'sentiment') 

**Are you running this notebook on an Amazon EC2 t2.micro instance?** (If you are using your own machine, please skip this section)

It has been reported that t2.micro instances do not provide sufficient power to complete the conversion in acceptable amount of time. For interest of time, please refrain from running `get_numpy_data` function. Instead, download the [binary file](https://s3.amazonaws.com/static.dato.com/files/coursera/course-3/numpy-arrays/module-3-assignment-numpy-arrays.npz) containing the four NumPy arrays you'll need for the assignment. To load the arrays, run the following commands:
```
arrays = np.load('module-3-assignment-numpy-arrays.npz')
feature_matrix, sentiment = arrays['feature_matrix'], arrays['sentiment']
```

In [30]:
feature_matrix.shape

(53072L, 194L)

** Quiz Question:** How many features are there in the **feature_matrix**?

** Quiz Question:** Assuming that the intercept is present, how does the number of features in **feature_matrix** relate to the number of features in the logistic regression model?

Now, let us see what the **sentiment** column looks like:

In [31]:
sentiment

array([ 1,  1,  1, ..., -1, -1, -1], dtype=int64)

## Estimating conditional probability with link function

Recall from lecture that the link function is given by:
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))},
$$

where the feature vector $h(\mathbf{x}_i)$ represents the word counts of **important_words** in the review  $\mathbf{x}_i$. Complete the following function that implements the link function:

In [40]:
'''
produces probablistic estimate for P(y_i = +1 | x_i, w).
estimate ranges between 0 and 1.
'''
def predict_probability(feature_matrix, coefficients):
    # Take dot product of feature_matrix and coefficients  
    # YOUR CODE HERE
    score = feature_matrix.dot(coefficients)
    # Compute P(y_i = +1 | x_i, w) using the link function
    # YOUR CODE HERE
    predictions = 1 / (1 + np.exp(-score))
    
    # return predictions
    return predictions

**Aside**. How the link function works with matrix algebra

Since the word counts are stored as columns in **feature_matrix**, each $i$-th row of the matrix corresponds to the feature vector $h(\mathbf{x}_i)$:
$$
[\text{feature_matrix}] =
\left[
\begin{array}{c}
h(\mathbf{x}_1)^T \\
h(\mathbf{x}_2)^T \\
\vdots \\
h(\mathbf{x}_N)^T
\end{array}
\right] =
\left[
\begin{array}{cccc}
h_0(\mathbf{x}_1) & h_1(\mathbf{x}_1) & \cdots & h_D(\mathbf{x}_1) \\
h_0(\mathbf{x}_2) & h_1(\mathbf{x}_2) & \cdots & h_D(\mathbf{x}_2) \\
\vdots & \vdots & \ddots & \vdots \\
h_0(\mathbf{x}_N) & h_1(\mathbf{x}_N) & \cdots & h_D(\mathbf{x}_N)
\end{array}
\right]
$$

By the rules of matrix multiplication, the score vector containing elements $\mathbf{w}^T h(\mathbf{x}_i)$ is obtained by multiplying **feature_matrix** and the coefficient vector $\mathbf{w}$.
$$
[\text{score}] =
[\text{feature_matrix}]\mathbf{w} =
\left[
\begin{array}{c}
h(\mathbf{x}_1)^T \\
h(\mathbf{x}_2)^T \\
\vdots \\
h(\mathbf{x}_N)^T
\end{array}
\right]
\mathbf{w}
= \left[
\begin{array}{c}
h(\mathbf{x}_1)^T\mathbf{w} \\
h(\mathbf{x}_2)^T\mathbf{w} \\
\vdots \\
h(\mathbf{x}_N)^T\mathbf{w}
\end{array}
\right]
= \left[
\begin{array}{c}
\mathbf{w}^T h(\mathbf{x}_1) \\
\mathbf{w}^T h(\mathbf{x}_2) \\
\vdots \\
\mathbf{w}^T h(\mathbf{x}_N)
\end{array}
\right]
$$

**Checkpoint**

Just to make sure you are on the right track, we have provided a few examples. If your `predict_probability` function is implemented correctly, then the outputs will match:

In [41]:
dummy_feature_matrix = np.array([[1.,2.,3.], [1.,-1.,-1]])
dummy_coefficients = np.array([1., 3., -1.])

correct_scores      = np.array( [ 1.*1. + 2.*3. + 3.*(-1.),          1.*1. + (-1.)*3. + (-1.)*(-1.) ] )
correct_predictions = np.array( [ 1./(1+np.exp(-correct_scores[0])), 1./(1+np.exp(-correct_scores[1])) ] )

print 'The following outputs must match '
print '------------------------------------------------'
print 'correct_predictions           =', correct_predictions
print 'output of predict_probability =', predict_probability(dummy_feature_matrix, dummy_coefficients)

The following outputs must match 
------------------------------------------------
correct_predictions           = [ 0.98201379  0.26894142]
output of predict_probability = [ 0.98201379  0.26894142]


## Compute derivative of log likelihood with respect to a single coefficient

Recall from lecture:
$$
\frac{\partial\ell}{\partial w_j} = \sum_{i=1}^N h_j(\mathbf{x}_i)\left(\mathbf{1}[y_i = +1] - P(y_i = +1 | \mathbf{x}_i, \mathbf{w})\right)
$$

We will now write a function that computes the derivative of log likelihood with respect to a single coefficient $w_j$. The function accepts two arguments:
* `errors` vector containing $\mathbf{1}[y_i = +1] - P(y_i = +1 | \mathbf{x}_i, \mathbf{w})$ for all $i$.
* `feature` vector containing $h_j(\mathbf{x}_i)$  for all $i$. 

Complete the following code block:

In [42]:
def feature_derivative(errors, feature):     
    # Compute the dot product of errors and feature
    derivative = errors.dot(feature)
    
    # Return the derivative
    return derivative

In the main lecture, our focus was on the likelihood.  In the advanced optional video, however, we introduced a transformation of this likelihood---called the log likelihood---that simplifies the derivation of the gradient and is more numerically stable.  Due to its numerical stability, we will use the log likelihood instead of the likelihood to assess the algorithm.

The log likelihood is computed using the following formula (see the advanced optional video if you are curious about the derivation of this equation):

$$\ell\ell(\mathbf{w}) = \sum_{i=1}^N \Big( (\mathbf{1}[y_i = +1] - 1)\mathbf{w}^T h(\mathbf{x}_i) - \ln\left(1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))\right) \Big) $$

We provide a function to compute the log likelihood for the entire dataset. 

In [43]:
def compute_log_likelihood(feature_matrix, sentiment, coefficients):
    indicator = (sentiment==+1)
    scores = np.dot(feature_matrix, coefficients)
    logexp = np.log(1. + np.exp(-scores))
    
    # Simple check to prevent overflow
    mask = np.isinf(logexp)
    logexp[mask] = -scores[mask]
    
    lp = np.sum((indicator-1)*scores - logexp)
    return lp

**Checkpoint**

Just to make sure we are on the same page, run the following code block and check that the outputs match.

In [44]:
dummy_feature_matrix = np.array([[1.,2.,3.], [1.,-1.,-1]])
dummy_coefficients = np.array([1., 3., -1.])
dummy_sentiment = np.array([-1, 1])

correct_indicators  = np.array( [ -1==+1,                                       1==+1 ] )
correct_scores      = np.array( [ 1.*1. + 2.*3. + 3.*(-1.),                     1.*1. + (-1.)*3. + (-1.)*(-1.) ] )
correct_first_term  = np.array( [ (correct_indicators[0]-1)*correct_scores[0],  (correct_indicators[1]-1)*correct_scores[1] ] )
correct_second_term = np.array( [ np.log(1. + np.exp(-correct_scores[0])),      np.log(1. + np.exp(-correct_scores[1])) ] )

correct_ll          =      sum( [ correct_first_term[0]-correct_second_term[0], correct_first_term[1]-correct_second_term[1] ] ) 

print 'The following outputs must match '
print '------------------------------------------------'
print 'correct_log_likelihood           =', correct_ll
print 'output of compute_log_likelihood =', compute_log_likelihood(dummy_feature_matrix, dummy_sentiment, dummy_coefficients)

The following outputs must match 
------------------------------------------------
correct_log_likelihood           = -5.33141161544
output of compute_log_likelihood = -5.33141161544


## Taking gradient steps

Now we are ready to implement our own logistic regression. All we have to do is to write a gradient ascent function that takes gradient steps towards the optimum. 

Complete the following function to solve the logistic regression model using gradient ascent:

In [48]:
from math import sqrt

def logistic_regression(feature_matrix, sentiment, initial_coefficients, step_size, max_iter):
    coefficients = np.array(initial_coefficients) # make sure it's a numpy array
    for itr in xrange(max_iter):

        # Predict P(y_i = +1|x_i,w) using your predict_probability() function
        # YOUR CODE HERE
        predictions = predict_probability(feature_matrix, coefficients)
        
        # Compute indicator value for (y_i = +1)
        indicator = (sentiment==+1)
        
        # Compute the errors as indicator - predictions
        errors = indicator - predictions
        for j in xrange(len(coefficients)): # loop over each coefficient
            
            # Recall that feature_matrix[:,j] is the feature column associated with coefficients[j].
            # Compute the derivative for coefficients[j]. Save it in a variable called derivative
            # YOUR CODE HERE
            derivative = feature_derivative(errors, feature_matrix[:,j])
            
            # add the step size times the derivative to the current coefficient
            ## YOUR CODE HERE
            coefficients[j] += step_size * derivative 
        
        # Checking whether log likelihood is increasing
        if itr <= 15 or (itr <= 100 and itr % 10 == 0) or (itr <= 1000 and itr % 100 == 0) \
        or (itr <= 10000 and itr % 1000 == 0) or itr % 10000 == 0:
            lp = compute_log_likelihood(feature_matrix, sentiment, coefficients)
            print 'iteration %*d: log likelihood of observed labels = %.8f' % \
                (int(np.ceil(np.log10(max_iter))), itr, lp)
    return coefficients

Now, let us run the logistic regression solver.

In [49]:
coefficients = logistic_regression(feature_matrix, sentiment, initial_coefficients=np.zeros(194),
                                   step_size=1e-7, max_iter=301)

iteration   0: log likelihood of observed labels = -36780.91768478
iteration   1: log likelihood of observed labels = -36775.13434712
iteration   2: log likelihood of observed labels = -36769.35713564
iteration   3: log likelihood of observed labels = -36763.58603240
iteration   4: log likelihood of observed labels = -36757.82101962
iteration   5: log likelihood of observed labels = -36752.06207964
iteration   6: log likelihood of observed labels = -36746.30919497
iteration   7: log likelihood of observed labels = -36740.56234821
iteration   8: log likelihood of observed labels = -36734.82152213
iteration   9: log likelihood of observed labels = -36729.08669961
iteration  10: log likelihood of observed labels = -36723.35786366
iteration  11: log likelihood of observed labels = -36717.63499744
iteration  12: log likelihood of observed labels = -36711.91808422
iteration  13: log likelihood of observed labels = -36706.20710739
iteration  14: log likelihood of observed labels = -36700.5020

In [50]:
coefficients

array([  5.16220157e-03,   1.55656966e-02,  -8.50204675e-03,
         6.65460842e-02,   6.58907629e-02,   5.01743882e-03,
        -5.38601484e-02,  -3.50488413e-03,   6.47945868e-02,
         4.54356263e-02,   3.98353364e-03,   2.00775410e-02,
         3.01350011e-02,  -2.87115530e-02,   1.52161964e-02,
         2.72592062e-04,   1.19448177e-02,  -1.82461935e-02,
        -1.21706420e-02,  -4.15110334e-02,   2.76820391e-03,
         1.77031999e-02,  -4.39700067e-03,   4.49764014e-02,
         9.90916464e-03,   8.99239081e-04,  -1.36219516e-03,
         1.26859357e-02,   8.26466695e-03,  -2.77426972e-02,
         6.10128809e-04,   1.54084501e-02,  -1.32134753e-02,
        -3.00512492e-02,   2.97399371e-02,   1.84087080e-02,
         2.86178752e-03,  -1.05768015e-02,  -6.57350362e-04,
        -1.01476555e-02,  -4.79579528e-03,   7.50891810e-03,
         4.27938289e-03,   3.06785501e-03,  -2.20317661e-03,
         9.57273354e-03,   9.91666827e-05,  -1.98462567e-02,
         1.75702722e-02,

**Quiz Question:** As each iteration of gradient ascent passes, does the log likelihood increase or decrease?

## Predicting sentiments

Recall from lecture that class predictions for a data point $\mathbf{x}$ can be computed from the coefficients $\mathbf{w}$ using the following formula:
$$
\hat{y}_i = 
\left\{
\begin{array}{ll}
      +1 & \mathbf{x}_i^T\mathbf{w} > 0 \\
      -1 & \mathbf{x}_i^T\mathbf{w} \leq 0 \\
\end{array} 
\right.
$$

Now, we will write some code to compute class predictions. We will do this in two steps:
* **Step 1**: First compute the **scores** using **feature_matrix** and **coefficients** using a dot product.
* **Step 2**: Using the formula above, compute the class predictions from the scores.

Step 1 can be implemented as follows:

In [51]:
# Compute the scores as a dot product between feature_matrix and coefficients.
scores = np.dot(feature_matrix, coefficients)

In [60]:
scores

array([ 0.05104571, -0.02936473,  0.02411584, ..., -0.40986295,
        0.01411436, -0.06755923])

Now, complete the following code block for **Step 2** to compute the class predictions using the **scores** obtained above:

In [69]:
predict_ = np.where(scores>0, +1, -1)
predict_

array([ 1, -1,  1, ..., -1,  1, -1])

** Quiz Question: ** How many reviews were predicted to have positive sentiment?

In [68]:
predict_[predict_ > 0].shape

(25126L,)

## Measuring accuracy

We will now measure the classification accuracy of the model. Recall from the lecture that the classification accuracy can be computed as follows:

$$
\mbox{accuracy} = \frac{\mbox{# correctly classified data points}}{\mbox{# total data points}}
$$

Complete the following code block to compute the accuracy of the model.

In [85]:
tmp = sentiment * predict_
num_mistakes = tmp[tmp < 0].shape[0]
accuracy = float((len(products) - num_mistakes)) / len(products)
print "-----------------------------------------------------"
print '# Reviews   correctly classified =', len(products) - num_mistakes
print '# Reviews incorrectly classified =', num_mistakes
print '# Reviews total                  =', len(products)
print "-----------------------------------------------------"
print 'Accuracy = %.2f' % accuracy

-----------------------------------------------------
# Reviews   correctly classified = 39903
# Reviews incorrectly classified = 13169
# Reviews total                  = 53072
-----------------------------------------------------
Accuracy = 0.75


**Quiz Question**: What is the accuracy of the model on predictions made above? (round to 2 digits of accuracy)

## Which words contribute most to positive & negative sentiments?

Recall that in Module 2 assignment, we were able to compute the "**most positive words**". These are words that correspond most strongly with positive reviews. In order to do this, we will first do the following:
* Treat each coefficient as a tuple, i.e. (**word**, **coefficient_value**).
* Sort all the (**word**, **coefficient_value**) tuples by **coefficient_value** in descending order.

In [86]:
coefficients = list(coefficients[1:]) # exclude intercept
word_coefficient_tuples = [(word, coefficient) for word, coefficient in zip(important_words, coefficients)]
word_coefficient_tuples = sorted(word_coefficient_tuples, key=lambda x:x[1], reverse=True)

Now, **word_coefficient_tuples** contains a sorted list of (**word**, **coefficient_value**) tuples. The first 10 elements in this list correspond to the words that are most positive.

### Ten "most positive" words

Now, we compute the 10 words that have the most positive coefficient values. These words are associated with positive sentiment.

In [89]:
word_coefficient_tuples

[('great', 0.066546084170457709),
 ('love', 0.065890762922123258),
 ('easy', 0.064794586802578394),
 ('little', 0.045435626308421365),
 ('loves', 0.044976401394906038),
 ('well', 0.030135001092107063),
 ('perfect', 0.029739937104968462),
 ('old', 0.020077541034775385),
 ('nice', 0.018408707995268992),
 ('daughter', 0.017703199905701694),
 ('soft', 0.017570272245602887),
 ('fits', 0.016882471071408715),
 ('happy', 0.01680529588976808),
 ('baby', 0.015565696580423503),
 ('recommend', 0.015408450108008665),
 ('also', 0.015216196422918844),
 ('best', 0.014991791565630267),
 ('comfortable', 0.013253990081584895),
 ('car', 0.012685935745813375),
 ('clean', 0.012018174433365525),
 ('son', 0.011944817713693949),
 ('bit', 0.01170824809312326),
 ('works', 0.011703160621987424),
 ('size', 0.010715966516270303),
 ('stroller', 0.0099091646359727326),
 ('room', 0.0097832410215680614),
 ('price', 0.0095727335435901798),
 ('play', 0.0091784289839843113),
 ('easily', 0.0090328181389544068),
 ('kids', 0

** Quiz Question:** Which word is **not** present in the top 10 "most positive" words?

### Ten "most negative" words

Next, we repeat this exercise on the 10 most negative words.  That is, we compute the 10 words that have the most negative coefficient values. These words are associated with negative sentiment.

In [90]:
word_coefficient_tuples[::-1]

[('would', -0.053860148445203142),
 ('product', -0.041511033392108897),
 ('money', -0.038982037286487116),
 ('work', -0.03306951529475273),
 ('even', -0.030051249236035801),
 ('disappointed', -0.028978976142317068),
 ('get', -0.028711552980192585),
 ('back', -0.027742697230661334),
 ('return', -0.026592778462247283),
 ('monitor', -0.02448210054589172),
 ('waste', -0.024042748071154956),
 ('thought', -0.021394348543682482),
 ('could', -0.01984625666077721),
 ('tried', -0.018702371424325837),
 ('way', -0.018359460662945689),
 ('time', -0.018246193486087036),
 ('buy', -0.017737543997218046),
 ('item', -0.017137867010854794),
 ('returned', -0.016001798500102516),
 ('broke', -0.01553769895565389),
 ('better', -0.015040658977043391),
 ('plastic', -0.014977044903587944),
 ('reviews', -0.01486631944997698),
 ('cheap', -0.014709833465080665),
 ('thing', -0.013868727265297582),
 ('hard', -0.013756676731261399),
 ('first', -0.013213475301677043),
 ('never', -0.012969046546319293),
 ('something', 

** Quiz Question:** Which word is **not** present in the top 10 "most negative" words?